<a href="https://colab.research.google.com/github/Geriys/VKirillV/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22NLP1_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Установка нужных пакетов
!pip install --upgrade nltk gensim bokeh umap-learn

import itertools
import string

import numpy as np
import umap
from nltk.tokenize import WordPunctTokenizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.3 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=75b95c60448bf9c62d131d79b959b3f414ab8199fb6213a16ad72de501a52fff
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
Successfully built umap-learn


In [2]:
# выгружаем датасет:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc

--2024-03-11 16:29:57--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2024-03-11 16:29:58--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc51d73eb44490797dcf658cabbc.dl.dropboxusercontent.com/cd/0/get/CO4o2YWnXx555h50LIUGO_gBluZvQjR7QZ_EYWd1U-odllxLQyevczBuNORzWuuLnTeRH2la7ee6pvehgPo3_w1kYflxVCbUghOKQngmBuL_vx8u8Y9JtAsdEvCmjG387YE/file?dl=1# [following]
--2024-03-11 16:29:58--  https://uc51d73eb44490797dcf658cabbc.dl.dropboxusercontent.com/cd/0/get/CO4o2YWnXx555h50LIUGO_gBluZvQjR7QZ_EYWd1U-odllxLQyevczBuNORzWuuLnTeRH2la7ee6pvehgPo3_w1kYflxVCbUghOKQngmBuL_vx8u8Y9JtAsdEvCmjG387Y

In [3]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

In [4]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


###Задание 1: Перевести все слова в нижний регистр (NLTK) из data и добавьте как лист токенов в листе data_tok


In [10]:
data_tok = [tokenizer.tokenize(line.lower()) for line in data]

print(data_tok[50])


#checking

assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"


['what', 'tv', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


###Задание 2: Подсчитайте топ10 самых популярных лем в рамках data

In [ ]:
# pip install nltk
python -m nltk.downloader punkt wordnet

In [17]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
# lemm = lemmatizer.lemmatize()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [18]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import Counter

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Загрузка данных из файла
data = list(open("./quora.txt", encoding="utf-8"))

# Инициализация лемматизатора
lemmatizer = WordNetLemmatizer()

# Токенизация и лемматизация текста
lemmatized_words = []
for line in data:
    tokens = word_tokenize(line.lower())
    lemmatized_words.extend([lemmatizer.lemmatize(token) for token in tokens])

# Подсчет частоты лем
word_freq = Counter(lemmatized_words)

# Получение топ-10 часто встречающихся лем
top_10_lemmas = word_freq.most_common(10)

# Вывод результата
for lemma, freq in top_10_lemmas:
    print(f"Lemma: {lemma}, Frequency: {freq}")

Lemma: ?, Frequency: 568407
Lemma: the, Frequency: 251847
Lemma: what, Frequency: 214452
Lemma: is, Frequency: 186142
Lemma: a, Frequency: 171580
Lemma: i, Frequency: 148983
Lemma: to, Frequency: 141488
Lemma: in, Frequency: 139213
Lemma: how, Frequency: 135423
Lemma: of, Frequency: 111930


###Задание 3: Подсчитайте количество разных слов до и после лемматизации

In [19]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Загрузка данных из файла
data = list(open("./quora.txt", encoding="utf-8"))

# Инициализация лемматизатора
lemmatizer = WordNetLemmatizer()

# Токенизация и лемматизация текста
lemmatized_words = []
original_words = []
for line in data:
    tokens = word_tokenize(line.lower())
    original_words.extend(tokens)
    lemmatized_words.extend([lemmatizer.lemmatize(token) for token in tokens])

# Количество уникальных слов до лемматизации
unique_original_words = len(set(original_words))

# Количество уникальных слов после лемматизации
unique_lemmatized_words = len(set(lemmatized_words))

# Вывод результата
print(f"Number of unique words before lemmatization: {unique_original_words}")
print(f"Number of unique words after lemmatization: {unique_lemmatized_words}")

Number of unique words before lemmatization: 112860
Number of unique words after lemmatization: 105352


###Задание 4: Подсчитайте количество разных слов до и после стемминга

###Задание 5: Подсчитайте количество разных слов



REGEXP

https://www.programiz.com/python-programming/regex

https://docs.python.org/3/howto/regex.html

In [ ]:
import re

pattern = 'a*s'
test_string = 'abyss'
result = re.match(pattern, test_string)

if result:
  print("Search successful.")
else:
  print("Search unsuccessful.")

Search unsuccessful.


### Отработка regex
###Задание 6:
https://www.hackerrank.com/challenges/matching-specific-string/problem?isFullScreen=true

###Задание 7:
https://www.hackerrank.com/challenges/matching-whitespace-non-whitespace-character/problem?isFullScreen=true

###Задание 8:
https://www.hackerrank.com/challenges/matching-start-end/problem?isFullScreen=true

###Задание 9:
https://www.hackerrank.com/challenges/matching-word-boundaries/problem?isFullScreen=true

Bag Of Words (BOW)

In [ ]:
def vectorize(tokens):
    ''' This function takes list of words in a sentence as input
    and returns a vector of size of filtered_vocab.It puts 0 if the
    word is not present in tokens and count of token if present.'''
    vector=[]
    for w in filtered_vocab:
        vector.append(tokens.count(w))
    return vector
def unique(sequence):
    '''This functions returns a list in which the order remains
    same and no item repeats.Using the set() function does not
    preserve the original ordering,so i didnt use that instead'''
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

#create a list of stopwords.You can import stopwords from nltk too
stopwords=["to","is","a"]

#list of special characters.You can use regular expressions too
special_char=[",",":"," ",";",".","?"]

#Write the sentences in the corpus,in our case, just two
string1="Welcome to Great Learning , Now start learning"
string2="Learning is a good practice"

#convert them to lower case
string1=string1.lower()
string2=string2.lower()

#split the sentences into tokens
tokens1=string1.split()
tokens2=string2.split()
print(tokens1)
print(tokens2)

#create a vocabulary list
vocab=unique(tokens1+tokens2)
print(vocab)

#filter the vocabulary list
filtered_vocab=[]
for w in vocab:
    if w not in stopwords and w not in special_char:
        filtered_vocab.append(w)
print(filtered_vocab)

#convert sentences into vectords
vector1=vectorize(tokens1)
print(vector1)
vector2=vectorize(tokens2)
print(vector2)

['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'learning']
['learning', 'is', 'a', 'good', 'practice']
['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'is', 'a', 'good', 'practice']
['welcome', 'great', 'learning', 'now', 'start', 'good', 'practice']
[1, 1, 2, 1, 1, 0, 0]
[0, 0, 1, 0, 0, 1, 1]


Задание 10: Реализовать Bag of words на data_tok (можно на NLTK, можно без)